In [44]:
# !pip install netcdf4

In [45]:
import pandas as pd
import numpy as np
import xarray as xr

In [46]:
# path = 'H:/My Drive/ArcGIS-NJ-HAND/Strahler'

# huc6 = '020200'
# file = path + '/' + huc6 + 'flow.csv'
# tables = pd.read_csv(file)


# for huc6 in ['020200', '020301', '020401', '020402', '020403']:
#     file = path + '/' + huc6 + 'flow.csv'
#     table = pd.read_csv(file)
#     tables = tables.append(table)

# tables

In [47]:
# tables.to_csv('./MapAvg/All_flowlines.csv',index=False)

In [48]:
# subset = tables[['COMID','StreamOrde','UpHydroseq','Hydroseq','DnHydroseq']]
# subset = tables[['COMID','StreamOrde','FromNode','ToNode']]
# subset.to_csv('./MapAvg/All_Sequence.csv',index=False)
# subset.head()

In [49]:
table = pd.read_csv('./MapAvg/All_Sequence.csv')
# table = subset.copy()
table.head()

,COMID,StreamOrde,FromNode,ToNode
0,22307225,6,200086447,200086448
1,22307227,6,200086448,200084661
2,22307221,6,200086445,200086446
3,22307205,6,200086437,200086440
4,22307211,6,200086440,200086445


In [50]:
xfile = './MapAvg/inun-hq-table-at-20200101_000000-for-20200101_000000.nc'
# xfile = './MapAvg/inun-hq-table-at-20200101_000000-for-20200101_000000_avg.nc'

datax = xr.open_dataset(xfile)
xr_att = datax.attrs
datax

<xarray.Dataset>
Dimensions:  (index: 11071)
Dimensions without coordinates: index
Data variables:
    COMID    (index) int64 ...
    H        (index) float64 ...
    Q        (index) float64 ...
Attributes:
    Subject:                   Inundation table derived from HAND and NOAA NW...
    Initialization_Timestamp:  20200101_000000
    Timestamp:                 20200101_000000
    Description:               Inundation lookup table for all the COMIDs in ...

In [51]:
datapd = datax.to_dataframe()
# datapd.reset_index()
print(len(datapd))
datapd.head(20)

11071


,COMID,H,Q
index,,,
0,6228092,0.028194,0.218375
1,6228094,0.172121,6.846027
2,6231300,0.370493,2.550000
3,6249670,2.135604,372.174840
4,6249686,5.017937,372.138780
5,6249688,1.402067,55.017543
6,6251062,1.282406,370.857567
7,6251012,1.010250,55.509122
8,6250998,1.303365,371.141810


In [52]:
# data2 = datapd.merge(table[['COMID','StreamOrde','UpHydroseq','Hydroseq','DnHydroseq']], on='COMID', how='left')
# # data2['UpH']  = ''
# # data2['UpOrde'] = ''
# # data2 = data2.merge(table[['COMID','ToNode']], on='COMID', how='left')
# # data2['DnH']  = ''
# # data2['DnOrde'] = ''
# # data2

# data2 = data2.merge(data2[['UpHydroseq','H']], left_on='Hydroseq', right_on='UpHydroseq', how='left')
# data2 = data2.rename(columns={'UpHydroseq_x': 'UpHydroseq', 'H_x': 'H', 'H_y': 'UpH'})
# # data2 = data2.drop(['UpHydroseq_y'], axis='columns')
# data2 = data2.merge(data2[['DnHydroseq','H']], left_on='Hydroseq', right_on='DnHydroseq', how='left')
# data2 = data2.rename(columns={'DnHydroseq_x': 'DnHydroseq', 'H_x': 'H', 'H_y': 'DnH'})
# # data2 = data2.drop(['DnHydroseq_y'], axis='columns')
# data2

data2 = datapd.merge(table, on='COMID', how='left')

data2 = data2.merge(data2[['FromNode','H','StreamOrde','ToNode']], left_on='ToNode', right_on='FromNode', how='left')
data2 = data2.rename(columns={'FromNode_x': 'FromNode', 'ToNode_x': 'ToNode',\
                              'StreamOrde_x': 'StreamOrde', 'StreamOrde_y': 'ToStreamOrde', \
                              'H_x': 'H', 'H_y': 'ToH', 'ToNode_y':'ToToNode'})
data2 = data2.drop(['FromNode_y'], axis='columns')

data2 = data2.merge(data2[['FromNode','H','StreamOrde']], left_on='ToToNode', right_on='FromNode', how='left')
data2 = data2.rename(columns={'FromNode_x': 'FromNode', 'ToNode_x': 'ToNode',\
                              'StreamOrde_x': 'StreamOrde', 'StreamOrde_y': 'ToToStreamOrde', \
                              'H_x': 'H', 'H_y': 'ToToH',}) # 'ToNode_y':'ToToNode'})
data2 = data2.drop(['FromNode_y'], axis='columns')

data2 = data2.merge(data2[['ToNode','H','StreamOrde','FromNode']], left_on='FromNode', right_on='ToNode', how='left')
data2 = data2.rename(columns={'ToNode_x': 'ToNode', 'FromNode_x':'FromNode', \
                              'StreamOrde_x': 'StreamOrde', 'StreamOrde_y': 'FromStreamOrde', \
                              'H_x': 'H', 'H_y': 'FromH', 'FromNode_y':'FromFromNode'})
data2 = data2.drop(['ToNode_y'], axis='columns')

data2 = data2.merge(data2[['ToNode','H','StreamOrde']], left_on='FromFromNode', right_on='ToNode', how='left')
data2 = data2.rename(columns={'ToNode_x': 'ToNode', 'FromNode_x':'FromNode', \
                              'StreamOrde_x': 'StreamOrde', 'StreamOrde_y': 'FromFromStreamOrde', \
                              'H_x': 'H', 'H_y': 'FromFromH',}) # 'FromNode_y':'FromFromNode'})
data2 = data2.drop(['ToNode_y'], axis='columns')

print(len(data2))
data2.head(20)

10396120


,COMID,H,Q,StreamOrde,FromNode,ToNode,ToH,ToStreamOrde,ToToNode,ToToH,ToToStreamOrde,FromH,FromStreamOrde,FromFromNode,FromFromH,FromFromStreamOrde
0,6228092,0.028194,0.218375,1,200105067,200031621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6228094,0.172121,6.846027,1,200031132,200031180,NaN,NaN,NaN,NaN,NaN,0.370493,1.0,200105212.0,NaN,NaN
2,6231300,0.370493,2.550000,1,200105212,200031132,0.172121,1.0,200031180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6249670,2.135604,372.174840,6,200032735,200033087,2.116199,6.0,200033086.0,3.268711,6.0,2.740965,6.0,200033090.0,1.895255,6.0
4,6249686,5.017937,372.138780,6,200032741,200033091,1.895255,6.0,200033090.0,2.740965,6.0,2.161597,6.0,200033101.0,2.138499,6.0
5,6249686,5.017937,372.138780,6,200032741,200033091,1.895255,6.0,200033090.0,2.740965,6.0,2.161597,6.0,200033101.0,2.138499,6.0
6,6249686,5.017937,372.138780,6,200032741,200033091,1.895255,6.0,200033090.0,2.740965,6.0,2.161597,6.0,200033101.0,1.093280,2.0
7,6249688,1.402067,55.017543,4,200032742,200033081,1.440535,4.0,200033083.0,1.010250,4.0,0.980474,3.0,200032674.0,1.905534,3.0
8,6249688,1.402067,55.017543,4,200032742,200033081,1.440535,4.0,200033083.0,1.010250,4.0,1.179165,3.0,200032675.0,1.373892,3.0
9,6249688,1.402067,55.017543,4,200032742,200033081,1.440535,4.0,200033083.0,1.010250,4.0,1.457376,1.0,200105518.0,NaN,NaN


In [53]:
test = data2.copy()
# test = test.dropna()
# test.dtypes
test['Avg3'] = ''
test['Avg5'] = ''
test.loc[test.FromStreamOrde == test.ToStreamOrde, 'Avg3' ] = (test.H + test.FromH + test.ToH) / 3
test.loc[test.FromFromStreamOrde == test.ToToStreamOrde, 'Avg5' ] = \
                    (test.FromFromH + test.FromH + test.H + test.ToH + test.ToToH) / 5
print(len(test))
test.head(20)

10396120


,COMID,H,Q,StreamOrde,FromNode,ToNode,ToH,ToStreamOrde,ToToNode,ToToH,ToToStreamOrde,FromH,FromStreamOrde,FromFromNode,FromFromH,FromFromStreamOrde,Avg3,Avg5
0,6228092,0.028194,0.218375,1,200105067,200031621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
1,6228094,0.172121,6.846027,1,200031132,200031180,NaN,NaN,NaN,NaN,NaN,0.370493,1.0,200105212.0,NaN,NaN,,
2,6231300,0.370493,2.550000,1,200105212,200031132,0.172121,1.0,200031180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
3,6249670,2.135604,372.174840,6,200032735,200033087,2.116199,6.0,200033086.0,3.268711,6.0,2.740965,6.0,200033090.0,1.895255,6.0,2.33092,2.43135
4,6249686,5.017937,372.138780,6,200032741,200033091,1.895255,6.0,200033090.0,2.740965,6.0,2.161597,6.0,200033101.0,2.138499,6.0,3.02493,2.79085
5,6249686,5.017937,372.138780,6,200032741,200033091,1.895255,6.0,200033090.0,2.740965,6.0,2.161597,6.0,200033101.0,2.138499,6.0,3.02493,2.79085
6,6249686,5.017937,372.138780,6,200032741,200033091,1.895255,6.0,200033090.0,2.740965,6.0,2.161597,6.0,200033101.0,1.093280,2.0,3.02493,
7,6249688,1.402067,55.017543,4,200032742,200033081,1.440535,4.0,200033083.0,1.010250,4.0,0.980474,3.0,200032674.0,1.905534,3.0,,
8,6249688,1.402067,55.017543,4,200032742,200033081,1.440535,4.0,200033083.0,1.010250,4.0,1.179165,3.0,200032675.0,1.373892,3.0,,
9,6249688,1.402067,55.017543,4,200032742,200033081,1.440535,4.0,200033083.0,1.010250,4.0,1.457376,1.0,200105518.0,NaN,NaN,,


In [54]:
sub1 = test[['COMID','H','Avg3']]
sub1.H = sub1.Avg3
sub1 = sub1.drop_duplicates(subset='COMID')
sub1.head(20)

,COMID,H,Avg3
0,6228092,,
1,6228094,,
2,6231300,,
3,6249670,2.33092,2.33092
4,6249686,3.02493,3.02493
7,6249688,,
10,6251062,,
11,6251012,1.18209,1.18209
14,6250998,1.98708,1.98708
15,6249824,3.37967,3.37967


In [55]:
sub2 = test[['COMID','H','Avg5']]
sub2.H = sub2.Avg5
sub2 = sub2.drop_duplicates(subset='COMID')
sub2.head(20)

# test.loc[test.Avg3 != '', 'H'] = test.loc[test.Avg3 != '', 'Avg3']
# test.loc[test.Avg5 != '', 'H'] = test.loc[test.Avg5 != '', 'Avg5']
# test.head(20)

,COMID,H,Avg5
0,6228092,,
1,6228094,,
2,6231300,,
3,6249670,2.43135,2.43135
4,6249686,2.79085,2.79085
7,6249688,,
10,6251062,,
11,6251012,1.24792,1.24792
14,6250998,2.39602,2.39602
15,6249824,,


In [56]:
# test = test.sort_values('H', ascending=True).drop_duplicates('COMID').sort_index()
# test = test.reset_index(drop=True)
# test

In [57]:
test.loc[test.COMID == 6249686]

,COMID,H,Q,StreamOrde,FromNode,ToNode,ToH,ToStreamOrde,ToToNode,ToToH,ToToStreamOrde,FromH,FromStreamOrde,FromFromNode,FromFromH,FromFromStreamOrde,Avg3,Avg5
4,6249686,5.017937,372.13878,6,200032741,200033091,1.895255,6.0,200033090.0,2.740965,6.0,2.161597,6.0,200033101.0,2.138499,6.0,3.02493,2.79085
5,6249686,5.017937,372.13878,6,200032741,200033091,1.895255,6.0,200033090.0,2.740965,6.0,2.161597,6.0,200033101.0,2.138499,6.0,3.02493,2.79085
6,6249686,5.017937,372.13878,6,200032741,200033091,1.895255,6.0,200033090.0,2.740965,6.0,2.161597,6.0,200033101.0,1.093280,2.0,3.02493,


In [58]:
# chop = test[['COMID','H','Q']]
chop = datapd.copy()
chop = chop.merge(sub1, on='COMID')
chop = chop.merge(sub2, on='COMID')
print(len(chop))
chop.head(20)

# chop.loc[5] - chop.loc[6]

11071


,COMID,H_x,Q,H_y,Avg3,H,Avg5
0,6228092,0.028194,0.218375,,,,
1,6228094,0.172121,6.846027,,,,
2,6231300,0.370493,2.550000,,,,
3,6249670,2.135604,372.174840,2.33092,2.33092,2.43135,2.43135
4,6249686,5.017937,372.138780,3.02493,3.02493,2.79085,2.79085
5,6249688,1.402067,55.017543,,,,
6,6251062,1.282406,370.857567,,,,
7,6251012,1.010250,55.509122,1.18209,1.18209,1.24792,1.24792
8,6250998,1.303365,371.141810,1.98708,1.98708,2.39602,2.39602
9,6249824,4.160703,371.521507,3.37967,3.37967,,


In [60]:
# chop = chop.drop_duplicates(subset='COMID')
# chop

In [61]:
chop.loc[chop.Avg3 != '', 'H_x'] = chop.loc[chop.Avg3 != '', 'Avg3']
chop.loc[chop.Avg5 != '', 'H_x'] = chop.loc[chop.Avg5 != '', 'Avg5']
chop

,COMID,H_x,Q,H_y,Avg3,H,Avg5
0,6228092,0.0281944,0.218375,,,,
1,6228094,0.172121,6.846027,,,,
2,6231300,0.370493,2.550000,,,,
3,6249670,2.43135,372.174840,2.33092,2.33092,2.43135,2.43135
4,6249686,2.79085,372.138780,3.02493,3.02493,2.79085,2.79085
5,6249688,1.40207,55.017543,,,,
6,6251062,1.28241,370.857567,,,,
7,6251012,1.24792,55.509122,1.18209,1.18209,1.24792,1.24792
8,6250998,2.39602,371.141810,1.98708,1.98708,2.39602,2.39602
9,6249824,3.37967,371.521507,3.37967,3.37967,,


In [17]:
flowx = chop.to_xarray()
flowx.attrs = xr_att
# flowx = flowx.drop_vars('index')
flowx

<xarray.Dataset>
Dimensions:  (index: 10396120)
Coordinates:
  * index    (index) int64 0 1 2 3 4 ... 10396116 10396117 10396118 10396119
Data variables:
    COMID    (index) int64 6228092 6228094 6231300 ... 9437361 9437359 9436501
    H        (index) float64 0.02819 0.1721 0.3705 ... 0.1376 0.03591 0.01625
    Q        (index) float64 0.2184 6.846 2.55 372.2 ... 1.476 0.3371 0.1433
Attributes:
    Subject:                   Inundation table derived from HAND and NOAA NW...
    Initialization_Timestamp:  20200101_000000
    Timestamp:                 20200101_000000
    Description:               Inundation lookup table for all the COMIDs in ...

In [18]:
# flowx.to_netcdf(xfile[:-3] + '_avg.nc')